In [9]:
import requests
import time
import os, json
import csv
import concurrent.futures
# from functools import partial
import pandas as pd

In [10]:
# url = "https://refract.fosfor.com/magiccodermodel/6188a3e2-bc43-444e-bfaa-8159fca71b3a/score"
url = "http://vllm-serving-service:8888/v1/completions"
# lltoken = "eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICJ4WTdTd3k5UE1xaXRDQmNSMm5qcVl6bmoxS3NqZzV3TmdOV0xDVzdyUkhvIn0.eyJleHAiOjE3MzA4ODM3MzgsImlhdCI6MTY5OTI2MTMzOCwiYXV0aF90aW1lIjoxNjk5MjUyMDQ0LCJqdGkiOiJmN2EzMzQwYy1kNDQwLTRlMzUtYjk2ZS04YzBiMTc0Y2RhODAiLCJpc3MiOiJodHRwczovL3JlZnJhY3QtbG9naW4uZm9zZm9yLmNvbS9hdXRoL3JlYWxtcy9tb3NhaWMiLCJhdWQiOlsibW9zYWljLWdhdGVrZWVwZXIiLCJhY2NvdW50Il0sInN1YiI6IjZjMjU4MWU3LWZmMTItNDljNy04MDJmLWI2ZjQzOWQxZDIwMSIsInR5cCI6IkJlYXJlciIsImF6cCI6Im1vc2FpYy1nYXRla2VlcGVyIiwic2Vzc2lvbl9zdGF0ZSI6IjBhY2Y3YWZhLTVmMzMtNGRhZS05OGM3LTQyZDQwYTdlZTM2NiIsImFsbG93ZWQtb3JpZ2lucyI6WyIqIl0sInJlYWxtX2FjY2VzcyI6eyJyb2xlcyI6WyJNTE9QUyIsImxvbmdfbGl2ZWRfdG9rZW4iLCJzcGVjdHJhLWRldmVsb3BlciIsImRlZmF1bHQtcm9sZXMtbW9zYWljIiwicmVmcmFjdC1kZXZlbG9wZXIiLCJvZmZsaW5lX2FjY2VzcyIsImFkbWluIiwidW1hX2F1dGhvcml6YXRpb24iLCJyZWZyYWN0LWFkbWluIl19LCJyZXNvdXJjZV9hY2Nlc3MiOnsiYWNjb3VudCI6eyJyb2xlcyI6WyJtYW5hZ2UtYWNjb3VudCIsIm1hbmFnZS1hY2NvdW50LWxpbmtzIiwidmlldy1wcm9maWxlIl19fSwic2NvcGUiOiJvcGVuaWQgZW1haWwgcHJvZmlsZSIsInNpZCI6IjBhY2Y3YWZhLTVmMzMtNGRhZS05OGM3LTQyZDQwYTdlZTM2NiIsImVtYWlsX3ZlcmlmaWVkIjp0cnVlLCJuYW1lIjoiUmVmcmFjdCBCRlNJIiwicHJlZmVycmVkX3VzZXJuYW1lIjoicmVmcmFjdC5iZnNpQGZvc2Zvci5jb20iLCJnaXZlbl9uYW1lIjoiUmVmcmFjdCIsImZhbWlseV9uYW1lIjoiQkZTSSIsImVtYWlsIjoicmVmcmFjdC5iZnNpQGZvc2Zvci5jb20ifQ.b6SYLgjo9Veo3GmJ8eZjCTNupQjpfMhzsoXdYjWwRtvRnNjBfx0gOqcugO9OcGn-mm8wwpSGI5uiL30-I6SdWBjsf1ur6GztoX7j-nP_3SrJJn3UhNNqIO8LbsPi5gGRTzWtnfjz92BF1YaCXxQwPY0P_aa8vJ6JxZz5Uctn9aIPIJZZnnjC_GPXtXurmshM_tEN2kwCjhEyr7wYzRqUoMtBGfpLjZREBzgZY-x6JyYiXNtycb1d6PFcCXf7nJVV8ienEC_x7OuciDzfeqd-SQnImvAHH7rqFdi9smBN08AbkDS2uAbMrokHrmbiBpaimrR013VwCWz2KL5QYlWleA"
request_payload_file = '/data/Magicoder_payloads.txt'
report_path = '/data/vLLM_15Concurrency_1gpu.csv'
df = pd.DataFrame(columns=['Prompt', 'Response', 'Response_status', 'Time (seconds)'])

In [11]:
payload = {"prompt": "Problem: Write a function in python to check if two strings are anagrams.Input: Strings s1 and s2.Output: Boolean, indicating whether s1 and s2 are anagrams.", 
           "max_tokens": 200,
           "temperature": 0,
           "model": "/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66"}

In [12]:
def read_payloads(file_path):
    with open(file_path, 'r') as file:
        for line in file:
            prompt = json.loads(line.strip())
            yield prompt['payload']

In [13]:
def my_decorator(func):
    def calculate_execution_time(*args, **kwargs):
        global df
        start_time = time.time()
        result, _= func(*args, **kwargs)
        end_time = time.time()
        execution_time = end_time - start_time
        data = {'Prompt':_,'Response':result.text, 'Response_status':result.status_code, 'Time (seconds)':f'{execution_time:.6f}'}
        df = df.append(data, ignore_index=True)
        print(f"Execution time: {execution_time:.6f} seconds \n\n")
        return result
    return calculate_execution_time

In [14]:
def save_report(file_path, df):
    print(df)
    df.to_csv(file_path, index=False)

In [15]:
@my_decorator
def inference(payload_data=None):
    headers = {
        'Content-Type': 'application/json'
    }
    try:
        response = requests.request("POST", url, headers=headers, json=payload_data)
        print(f"Prompt==> {payload_data['prompt']} \n {response.text}")
        return response, payload_data['prompt']
    except Exception as e:
        print(e)
        

In [ ]:
#Model Inference(Request)
if __name__ == "__main__":
    # Using the generator function to read lines 
#     prompts = read_payloads(request_payload_file)
    
    global df
    for i in range(15):
        prompts = read_payloads(request_payload_file)
        print(f"loop {i}+++++++++++++++++++++++++++++")
        with concurrent.futures.ThreadPoolExecutor(max_workers=15) as executor:
            futures = [executor.submit(inference, {**payload, 'prompt': prompt}) for prompt in prompts]
            concurrent.futures.wait(futures)
    save_report(report_path, df)

loop 0+++++++++++++++++++++++++++++
Prompt==> Problem: Write a function in python to reverse a given string.Input: String s.Output: String, the reversed version of s. 
 {"id":"cmpl-99f8a3069d35470294e7cbddd18c1067","object":"text_completion","created":447088,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\n\ndef reverse_string(s):\n    return s[::-1]\n\n# Test the function\nprint(reverse_string(\"Hello, World!\"))  # Output: !dlroW ,olleH","logprobs":null,"finish_reason":"stop"}],"usage":{"prompt_tokens":31,"total_tokens":82,"completion_tokens":51}}
Execution time: 3.608346 seconds 


Prompt==> Problem: Write a function in python to count the number of words in a given string.Input: String s.Output: Integer, the number of words in s. 
 {"id":"cmpl-88c5a3bb7c364581b8f1d029cf4baff1","object":"text_completion","created":447088,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/c

Prompt==> Problem: Write a function in java to calculate the factorial of a given non-negative integer.Input: Non-negative integer n. Output: Integer, the factorial of n. 
 {"id":"cmpl-8cee5bf3ea4243b1be862a50467a47f9","object":"text_completion","created":447088,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\n\nSolution:\n\nHere is a simple Java function to calculate the factorial of a given non-negative integer:\n\n```java\npublic class Main {\n    public static void main(String[] args) {\n        System.out.println(factorial(5));  // Output: 120\n    }\n\n    public static int factorial(int n) {\n        if (n == 0) {\n            return 1;\n        } else {\n            return n * factorial(n - 1);\n        }\n    }\n}\n```\n\nThis function uses recursion to calculate the factorial. The base case is when `n` is 0, in which case the function returns 1. For all other values of `n`, the func

Prompt==> Problem: Write a function in c++ to reverse a given string.Input: String s.Output: String, the reversed version of s. 
 {"id":"cmpl-3520d33bffe04f4bb08883103f412bd6","object":"text_completion","created":447096,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\n\n#include <iostream>\n#include <string>\n\nusing namespace std;\n\nstring reverseString(string s) {\n    int len = s.length();\n    string reversed = \"\";\n    for (int i = len - 1; i >= 0; i--) {\n        reversed += s[i];\n    }\n    return reversed;\n}\n\nint main() {\n    string s;\n    cout << \"Enter a string: \";\n    getline(cin, s);\n    cout << \"Reversed string: \" << reverseString(s) << endl;\n    return 0;\n}","logprobs":null,"finish_reason":"stop"}],"usage":{"prompt_tokens":32,"total_tokens":180,"completion_tokens":148}}
Execution time: 9.189515 seconds 


Prompt==> Problem: Write a function in java to perform bi

Prompt==> Problem: Write a function in c++ to calculate the sum of the digits of a given positive integer.Input: Positive integer num.Output: Integer, the sum of the digits of num. 
 {"id":"cmpl-cddf0ebd2c274bbf8a8761a381cdf051","object":"text_completion","created":447100,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\n\nSolution:\n\n```cpp\n#include <iostream>\nusing namespace std;\n\nint sumOfDigits(int num) {\n    int sum = 0;\n    while (num != 0) {\n        sum += num % 10;\n        num /= 10;\n    }\n    return sum;\n}\n\nint main() {\n    int num;\n    cout << \"Enter a positive integer: \";\n    cin >> num;\n    cout << \"Sum of digits: \" << sumOfDigits(num) << endl;\n    return 0;\n}\n```\n\nThis program first asks the user to input a positive integer. It then calls the function `sumOfDigits` with the input number as the argument. The function calculates the sum of the digits by re

Prompt==> Problem: Write a function in Ruby to calculate the factorial of a given non-negative integer.Input: Non-negative integer n. Output: Integer, the factorial of n. 
 {"id":"cmpl-3cc434b4596b4406979d9a59287e77e9","object":"text_completion","created":447112,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\n\ndef factorial(n)\n  if n == 0\n    1\n  else\n    n * factorial(n-1)\n  end\nend\n\nputs factorial(5)","logprobs":null,"finish_reason":"stop"}],"usage":{"prompt_tokens":41,"total_tokens":88,"completion_tokens":47}}
Execution time: 3.181151 seconds 


Prompt==> Problem: Write a function in Ruby to check if a given string is a palindrome.Input: String s.Output: Boolean, indicating whether s is a palindrome. 
 {"id":"cmpl-c312275b16f042d38bc819f717e58770","object":"text_completion","created":447113,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c37

...

Prompt==> Problem: Write a function in java to check if two strings are anagrams.Input: Strings s1 and s2.Output: Boolean, indicating whether s1 and s2 are anagrams. 
 {"id":"cmpl-cfe16e9483f345218fcd5480a9ca7d31","object":"text_completion","created":447168,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\nAn anagram is a word or phrase formed by rearranging the letters of a different word or phrase, typically using all the original letters exactly once.\n\nHere is a simple Java function to check if two strings are anagrams:\n\n```java\nimport java.util.Arrays;\n\npublic class Main {\n    public static void main(String[] args) {\n        System.out.println(areAnagrams(\"listen\", \"silent\"));  // Output: true\n        System.out.println(areAnagrams(\"hello\", \"world\"));  // Output: false\n    }\n\n    public static boolean areAnagrams(String s1, String s2) {\n        if (s1.length() != s2.len

Prompt==> Problem: Write a function in javascript to calculate the factorial of a given non-negative integer.Input: Non-negative integer n. Output: Integer, the factorial of n. 
 {"id":"cmpl-9b8b51c6d3254396a12f00aacb2ca30a","object":"text_completion","created":447180,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\n\nfunction factorial(n) {\n    if (n === 0) {\n        return 1;\n    } else {\n        return n * factorial(n - 1);\n    }\n}\n\nconsole.log(factorial(5)); // Output: 120","logprobs":null,"finish_reason":"stop"}],"usage":{"prompt_tokens":40,"total_tokens":107,"completion_tokens":67}}
Execution time: 4.237613 seconds 


Prompt==> Problem: Write a function in javascript to check if a given string is a palindrome.Input: String s.Output: Boolean, indicating whether s is a palindrome. 
 {"id":"cmpl-2c0538822907416a9f68bee48b45a425","object":"text_completion","created":447181,"model":"

Prompt==> Problem: Write a function in Ruby to reverse a given string.Input: String s.Output: String, the reversed version of s. 
 {"id":"cmpl-921e0e6fb6df45dea716be3a5d51370b","object":"text_completion","created":447188,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\n\ndef reverse_string(s)\n  s.reverse\nend\n\nputs reverse_string(\"Hello World\")","logprobs":null,"finish_reason":"stop"}],"usage":{"prompt_tokens":32,"total_tokens":62,"completion_tokens":30}}
Execution time: 2.149696 seconds 


Prompt==> Problem: Write a function in javascript to count the number of words in a given string.Input: String s.Output: Integer, the number of words in s. 
 {"id":"cmpl-56b244ca9a39425c9bc596f0387c4c79","object":"text_completion","created":447183,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\n\nfunction c

Prompt==> Problem: Write a function in Ruby to transpose a given matrix.Input: 2D list representing a matrix.Output: 2D list, the transpose of the input matrix. 
 {"id":"cmpl-04fc4ff2b06c45f1a76cc34cf7247279","object":"text_completion","created":447190,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\n\ndef transpose(matrix)\n  matrix.transpose\nend\n\n# Test the function\nmatrix = [[1, 2, 3], [4, 5, 6], [7, 8, 9]]\nputs \"Original Matrix:\"\nmatrix.each { |row| puts row.join(' ') }\n\ntransposed_matrix = transpose(matrix)\nputs \"Transposed Matrix:\"\ntransposed_matrix.each { |row| puts row.join(' ') }","logprobs":null,"finish_reason":"stop"}],"usage":{"prompt_tokens":42,"total_tokens":158,"completion_tokens":116}}
Execution time: 7.417424 seconds 


Prompt==> Problem: Write a function in javascript to transpose a given matrix.Input: 2D list representing a matrix.Output: 2D list, the transpos

Prompt==> Problem: Write a function in Golang to reverse a given string.Input: String s.Output: String, the reversed version of s. 
 {"id":"cmpl-915eb715c5f54e4aaae9b0f1ba1c2ef7","object":"text_completion","created":447193,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\n\n```go\npackage main\n\nimport (\n\t\"fmt\"\n\t\"strings\"\n)\n\nfunc reverseString(s string) string {\n\trunes := []rune(s)\n\tfor i, j := 0, len(runes)-1; i < j; i, j = i+1, j-1 {\n\t\trunes[i], runes[j] = runes[j], runes[i]\n\t}\n\treturn string(runes)\n}\n\nfunc main() {\n\ts := \"Hello, World!\"\n\tfmt.Println(reverseString(s))\n}\n```\n\nThis program defines a function `reverseString` that takes a string `s` as input and returns the reversed string. It does this by converting the string to a slice of runes (which are the equivalent of characters in Go), then swapping the","logprobs":null,"finish_reason":"length"}],"usa

In [ ]:
# import aiohttp
# import asyncio
# import time

# start_time = time.time()
# headers={"Content-type":"application/json"} 
# async def main():

#     async with aiohttp.ClientSession(trust_env = True, headers=headers) as session:

# #         for payload in payloads:
# #             url='https://refract.fosfor.com/openchatfinetunedimqlgenerator/d0721c53-c9f5-4d79-a988-2d21df46198f/score' 
#         async with session.post(url, json=payload) as resp:
#                 mql = await resp.json(content_type=None)
#                 print(mql)

# await main()
# print("--- %s seconds ---" % (time.time() - start_time))